In [1]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import torch
from omegaconf import OmegaConf

2023-05-15 12:30:37.543206: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-15 12:30:37.593280: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-15 12:30:37.595297: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
SEED = 27

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [5]:
conf = OmegaConf.load("config.yaml") #get config

In [25]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
tokenizer = GPT2TokenizerFast.from_pretrained(conf.model.model_name_or_path)
special_tokens_dict = {'additional_special_tokens': ['user:', 'bot:']}
tokenizer.add_special_tokens({'eos_token': '<instructionE>'})
tokenizer.add_special_tokens({'bos_token': '<instructionE>'})
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.add_special_tokens(special_tokens_dict)
model = GPT2LMHeadModel.from_pretrained('/mnt/opt/alexw/Kar/chatbot_saiga_dataset/boseos_saiga+my_2')
model.to(device)
model.resize_token_embeddings(len(tokenizer))

Embedding(50266, 1536)

In [7]:
tokenizer.padding_side = 'left'

In [7]:
print(tokenizer.decode([50264]))

user:


In [10]:
print(tokenizer.encode('\n'))

[203]


In [69]:
print(tokenizer.eos_token_id)

50261


In [90]:
def controlled_generation(prompt, word, threshold=0.7):
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    output = model.generate(
        input_ids=input_ids,
        min_length=20,
        max_new_tokens=520,
        do_sample=True,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        top_k=70,
        top_p=0.6,
        temperature=1.2,
        no_repeat_ngram_size=2,
        bad_words_ids=[[225], [50263], [50264], [50265], [203]],
        num_return_sequences=1
    )
    print(tokenizer.decode(output[0]))
    # generated_text = tokenizer.decode(output[0])
    
    input_ids_word = tokenizer.encode(word, return_tensors='pt').to(device)
    input_word = tokenizer.encode(tokenizer.eos_token + ' ' + word, return_tensors='pt').to(device)
    prompt_embedding = model.transformer.wte(input_ids_word).mean(dim=1)
    output_all = torch.cat([input_word, output], dim=-1)
    # output_all = output
    cosine_sim_after = -1
    for i in range(len(output_all[0])):
        next_token = output_all[:, i]
        next_token_embedding = model.transformer.wte(next_token).mean(dim=1)
        cosine_sim = torch.nn.functional.cosine_similarity(prompt_embedding, next_token_embedding, dim=-1)
        
        
        if cosine_sim.item() < threshold and output_all[0, i] == 50261 and i > (len(input_ids[0])+ len(input_ids_word[0])):
            while cosine_sim_after <= cosine_sim:
            # print(output[:, :i])
                print('before: ', cosine_sim.item())
                next_new_token = model.generate(
                    input_ids=output_all[:, :i],
                    bos_token_id=tokenizer.bos_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.pad_token_id,
                    do_sample=True,
                    max_new_tokens=150,
                    top_k=70,
                    top_p=0.6,
                    temperature=1.2,
                    no_repeat_ngram_size=2,
                    bad_words_ids=[[225], [50263], [50264], [50265]],
                    num_return_sequences=1
                )
            # print(next_token)
                next_token_embedding = model.transformer.wte(next_new_token).mean(dim=1)
                cosine_sim_after = torch.nn.functional.cosine_similarity(prompt_embedding, next_token_embedding, dim=-1)
                print(tokenizer.decode(next_new_token[0]))
                print('after: ', cosine_sim_after.item())
            
            output_all = next_new_token
            # output[0, i] = next_token
        elif cosine_sim.item() < threshold and i > (len(input_ids[0])+ len(input_ids_word[0])):#(len(input_ids[0])+ len(input_ids_word[0]))
            # print(abs(cosine_sim.item()))
            while cosine_sim_after <= cosine_sim:
                print('output ', tokenizer.decode(output_all[0, :i]))
                print('before: ', cosine_sim.item())
                next_new_token = model.generate(
                    input_ids=output_all[:, :i],
                    max_length=output_all.shape[-1] + 1,
                    bos_token_id=tokenizer.bos_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.pad_token_id,
                    do_sample=True,
                    min_length=1,
                    top_k=100,
                    top_p=0.2,
                    temperature=1.2,
                    no_repeat_ngram_size=1,
                    bad_words_ids=[[225], [50263], [50264], [50265]],
                    num_return_sequences=1
                )[:, i]
            # print(next_token)
                next_token_embedding = model.transformer.wte(next_new_token).mean(dim=1)
                cosine_sim_after = torch.nn.functional.cosine_similarity(prompt_embedding, next_token_embedding, dim=-1)
                print(tokenizer.decode(next_new_token[0]))
                print('after: ', cosine_sim_after.item())
                # output = torch.cat([output, next_token.unsqueeze(0)], dim=-1)
            output_all[0, i] = next_new_token
            
                
        
    generated_text = tokenizer.decode(output_all[0])

    return generated_text

In [190]:
def controlled_generation(prompt, word):
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    output = model.generate(
        input_ids=input_ids,
        min_length=20,
        max_new_tokens=520,
        do_sample=True,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        top_k=70,
        top_p=0.6,
        temperature=1.2,
        no_repeat_ngram_size=2,
        bad_words_ids=[[225], [50263], [50264], [50265], [203]],
        num_return_sequences=1
    )
    print(tokenizer.decode(output[0]))
    
    input_ids_word = tokenizer.encode(word, return_tensors='pt').to(device)
    prompt_embedding = model.transformer.wte(input_ids_word).mean(dim=1)
    
    input_word = tokenizer.encode(tokenizer.eos_token + ' ' + word, return_tensors='pt').to(device)
    # input_all = torch.cat([input_word, input_ids], dim=-1)
    input_all = input_ids
    next_token_embedding = model.transformer.wte(output).mean(dim=1)
    cosine_sim = torch.nn.functional.cosine_similarity(prompt_embedding, next_token_embedding, dim=-1)
    cosine_sim_after = cosine_sim
    bad_output = torch.cat([input_ids, torch.tensor([[tokenizer.eos_token_id]]).to(device)], dim=-1)
    next_new_token = bad_output
    
    if cosine_sim.item() < 0:   
        while cosine_sim_after.item() <= cosine_sim.item() or tokenizer.decode(next_new_token[0]) == tokenizer.decode(bad_output[0]):
            next_new_token = model.generate(
                input_ids=input_all,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=True,
                max_new_tokens=150,
                top_k=100,
                top_p=0.2,
                temperature=1.2,
                no_repeat_ngram_size=2,
                bad_words_ids=[[225], [50263], [50264], [50265]],
                num_return_sequences=1
            )
            next_token_embedding = model.transformer.wte(next_new_token).mean(dim=1)
            cosine_sim_after = torch.nn.functional.cosine_similarity(prompt_embedding, next_token_embedding, dim=-1)
            
    output_all = next_new_token   
                    
    generated_text = tokenizer.decode(output_all[0])

    return generated_text

In [192]:

prompt = "<instructionE> user: что ты делаешь? bot: <instructionE>"
generated_text = controlled_generation(prompt, word="ты ребенок")
print(generated_text)


<instructionE> user: что ты делаешь? bot: <instructionE>Я читаю книги и смотрю фильмы. А ты?<instructionE>
<instructionE> user: что ты делаешь? bot: <instructionE>Я занимаюсь разработкой программного обеспечения.<instructionE>


In [26]:
gen_kwargs = {
        "max_new_tokens": 100,
        "top_k": 70,
        "top_p": 0.8,
        "do_sample": True,  
        # "early_stopping": True,
        "no_repeat_ngram_size": 2,
        "bos_token_id": tokenizer.bos_token_id,
        "eos_token_id": tokenizer.eos_token_id,
        "pad_token_id": tokenizer.pad_token_id,
        "temperature": 0.8,
        "use_cache": True,
        "repetition_penalty": 1.2,  
        "bad_words_ids": [[225], [50263], [50264], [50265], [203]],
        "num_return_sequences": 1
    }
min_length=20
context = ''
inp = ''
while True:
    inp = input(">>> ")
    if inp == "stop":
        break
    inp_prepared = context + ' user: ' + inp + ' bot: <instructionE> '
    prepared = tokenizer.encode(inp_prepared, return_tensors='pt').to(device)
    out = model.generate(min_length=int(min_length+len(context)*0.5), input_ids=prepared, **gen_kwargs)
    generated = tokenizer.decode(out[0])
    print(generated)
    # print(gn_result)
    
    context = generated
    # if min_length > 480:
    # print(context.split('user:'))

>>>  как вывести график на python?


 user: как вывести график на python? bot: <instructionE> `your_image()` позволяет выводить координаты в графике на экране. Чтобы вывести этот график, нужно использовать функцию set(). Она возвращает значение параметра функции, которая принимает аргумент. Например: 
``pythоны = {'name', 'John', 30};`. Здесь мы передаем функцию и выводим ее на экран с помощью функции get(), что означает "John". Если вы хотите получить координаты от 1 до 5 точек, можно передать их в функцию


KeyboardInterrupt: Interrupted by user

In [130]:
out[0]

tensor([50261,   225, 50264,   417, 41213,  1642,   282,   515,  6784,   225,
        50265,   225,     5,  9230,   329,  1304,  4162,    35,   889,   895,
          953,  9245,   281, 30924, 18183,   289, 44437,  1163,   979, 50261],
       device='cuda:0')

In [116]:
tokenizer.decode([50261])

'<instructionE>'